## Test debug

In [3]:
import json
import pprint
import numpy as np
import matplotlib.pyplot as plt
import os

In [4]:
def calculate_error_dict(file_path):
    with open(file_path, 'r') as f:
        test_debug = json.load(f)
    
    (total_error, total_count) = (0, 0)
    error_dict = {v['edfname']: {} for v in test_debug.values()}
    
    for v in test_debug.values():
        gt = v['GT']
        pred = v['Pred']
        errors = sum([p for i, p in enumerate(pred) if i != gt])
        edfname = v['edfname']

        error_dict[edfname]['counts'] = error_dict[edfname].get('counts', 0) + 1
        error_dict[edfname]['errors'] = error_dict[edfname].get('errors', 0) + errors
        
        total_error += errors
        total_count += 30

        if gt == 0:
            error_dict[edfname]['color'] = 'green'
            error_dict[edfname]['label'] = 'Normal'
        elif gt == 1:
            error_dict[edfname]['color'] = 'blue'
            error_dict[edfname]['label'] = 'Non-vascular MCI'
        else:
            error_dict[edfname]['color'] = 'red'
            error_dict[edfname]['label'] = 'Non-vascular dementia'
    error_dict['accuracy'] = (1 - total_error / total_count) * 100
    return error_dict

In [5]:
file_path = r'C:\Users\IPIS-Minjae\Desktop\EEG_Project\test_debug\Original_0.5s_TinyCNN.txt'
original_test_set = set(calculate_error_dict(file_path).keys())

file_path = r'C:\Users\IPIS-Minjae\Desktop\EEG_Project\test_debug\Segment_0.5s_TinyCNN.txt'
segmented_test_set = set(calculate_error_dict(file_path).keys())

joint_test_set = original_test_set & segmented_test_set
joint_test_set.discard('accuracy')
print(len(original_test_set), len(segmented_test_set), len(joint_test_set))

105 960 93


## 1D CNNs

In [6]:
plt.style.use('fivethirtyeight') # default, ggplot, fivethirtyeight, classic

for seconds in [0.5, 1, 1.5, 2, 3, 5, 10, 15, 60]:
    debug_table = {}
    
    for network in ['TinyCNN', 'M5', 'ResNet']: # ['TinyCNN', 'M5', 'ResNet', 'TinyResNet']:
        file_path = f'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/Original_{seconds}s_{network}.txt'
        debug_table[f'1D {network} (Original {seconds}s)'] = calculate_error_dict(file_path)
    
    fig = plt.figure(num=1, clear=True, figsize=(40.0, 15.0), constrained_layout=True)

    N = len(debug_table)
    for i, (title, error_dict) in enumerate(debug_table.items()):
        ax = fig.add_subplot(N, 1, i + 1)

        for k, v in sorted(error_dict.items()):
            if k != 'accuracy':
                prob = v['errors'] / (v['counts'] * 30)
                rect = ax.bar(k, prob, color=v['color'])
                # ax.bar_label(rect, padding=3)    

        ax.set_title(f'{title} {error_dict["accuracy"]:.2f}%', fontsize=30)
        ax.set_ylim(0.0, 1.0)
        
        if i < N - 1:
            plt.setp(ax.get_xticklabels(), visible=False)
        else:
            plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)
            
    out_path = 'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/imgs/'
    os.makedirs(out_path, exist_ok=True)
    fig.savefig(os.path.join(out_path, f'original_{seconds}s.png'))
    
    # plt.show()
    fig.clear()
    plt.close(fig)

In [7]:
plt.style.use('fivethirtyeight') # default, ggplot, fivethirtyeight, classic

for seconds in [0.5, 1, 1.5, 2, 3, 5, 10, 15, 60]:
    debug_table = {}
    
    for network in ['TinyCNN', 'M5', 'ResNet', 'TinyResNet']:
        file_path = f'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/Original_{seconds}s_{network}.txt'
        debug_table[f'1D {network} (Original {seconds}s) - Joint'] = calculate_error_dict(file_path)
    
    fig = plt.figure(num=1, clear=True, figsize=(40.0, 15.0), constrained_layout=True)

    N = len(debug_table)
    for i, (title, error_dict) in enumerate(debug_table.items()):
        ax = fig.add_subplot(N, 1, i + 1)

        for k, v in sorted(error_dict.items()):
            if k in joint_test_set:
                prob = v['errors'] / (v['counts'] * 30)
                rect = ax.bar(k, prob, color=v['color'])
                # ax.bar_label(rect, padding=3)    

        ax.set_title(f'{title} {error_dict["accuracy"]:.2f}%', fontsize=30)
        ax.set_ylim(0.0, 1.0)
        
        if i < N - 1:
            plt.setp(ax.get_xticklabels(), visible=False)
        else:
            plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)


    out_path = 'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/imgs/'
    os.makedirs(out_path, exist_ok=True)
    fig.savefig(os.path.join(out_path, f'original_{seconds}s_joint.png'))
    
    # plt.show()
    fig.clear()
    plt.close(fig)

## 1D CNNs (Segmented)

In [8]:
plt.style.use('fivethirtyeight') # default, ggplot, fivethirtyeight, classic

for seconds in [0.5, 1, 1.5, 2, 3, 5, 10, 15]:
    debug_table = {}
    
    for network in ['TinyCNN', 'M5', 'ResNet', 'TinyResNet']:
        file_path = f'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/Segment_{seconds}s_{network}.txt'
        debug_table[f'1D {network} (Segmented {seconds}s)'] = calculate_error_dict(file_path)
    
    fig = plt.figure(num=1, clear=True, figsize=(120.0, 45.0), constrained_layout=True)

    N = len(debug_table)
    for i, (title, error_dict) in enumerate(debug_table.items()):
        ax = fig.add_subplot(N, 1, i + 1)

        for k, v in sorted(error_dict.items()):
            if k != 'accuracy':
                prob = v['errors'] / (v['counts'] * 30)
                rect = ax.bar(k, prob, color=v['color'])
                # ax.bar_label(rect, padding=3)    

        ax.set_title(f'{title} {error_dict["accuracy"]:.2f}%', fontsize=90)
        ax.set_ylim(0.0, 1.0)
        
        if i < N - 1:
            plt.setp(ax.get_xticklabels(), visible=False)
        else:
            plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)


    out_path = 'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/imgs/'
    os.makedirs(out_path, exist_ok=True)
    fig.savefig(os.path.join(out_path, f'segmented_{seconds}s.png'))

    # plt.show()
    fig.clear()
    plt.close(fig)

In [9]:
plt.style.use('fivethirtyeight') # default, ggplot, fivethirtyeight, classic

for seconds in [0.5, 1, 1.5, 2, 3, 5, 10, 15]:
    debug_table = {}
    
    for network in ['TinyCNN', 'M5', 'ResNet', 'TinyResNet']:
        file_path = f'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/Segment_{seconds}s_{network}.txt'
        debug_table[f'1D {network} (Segmented {seconds}s) - Joint'] = calculate_error_dict(file_path)
    
    fig = plt.figure(num=1, clear=True, figsize=(40.0, 15.0), constrained_layout=True)

    N = len(debug_table)
    for i, (title, error_dict) in enumerate(debug_table.items()):
        ax = fig.add_subplot(N, 1, i + 1)

        for k, v in sorted(error_dict.items()):
            if k in joint_test_set:
                prob = v['errors'] / (v['counts'] * 30)
                rect = ax.bar(k, prob, color=v['color'])
                # ax.bar_label(rect, padding=3)    

        ax.set_title(f'{title} {error_dict["accuracy"]:.2f}%', fontsize=30)
        ax.set_ylim(0.0, 1.0)
        
        if i < N - 1:
            plt.setp(ax.get_xticklabels(), visible=False)
        else:
            plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)

    out_path = 'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/imgs/'
    os.makedirs(out_path, exist_ok=True)
    fig.savefig(os.path.join(out_path, f'segmented_{seconds}s_joint.png'))

    # plt.show()
    fig.clear()
    plt.close(fig)

## 1D CNNs (60s)

In [10]:
plt.style.use('fivethirtyeight') # default, ggplot, fivethirtyeight, classic

for seconds in [60]:
    debug_table = {}
    
    for network in ['TinyCNN', 'M5' , 'M5_no_age', 'ResNet', 'ResNet_no_age', 'TinyResNet']:
        file_path = f'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/Original_{seconds}s_{network}.txt'
        debug_table[f'2D {network} ({seconds}s)'] = calculate_error_dict(file_path)
    
    fig = plt.figure(num=1, clear=True, figsize=(40.0, 22.5), constrained_layout=True)

    N = len(debug_table)
    for i, (title, error_dict) in enumerate(debug_table.items()):
        ax = fig.add_subplot(N, 1, i + 1)

        for k, v in sorted(error_dict.items()):
            if k != 'accuracy':
                prob = v['errors'] / (v['counts'] * 30)
                rect = ax.bar(k, prob, color=v['color'])
                # ax.bar_label(rect, padding=3)    

        ax.set_title(f'{title} {error_dict["accuracy"]:.2f}%', fontsize=30)
        ax.set_ylim(0.0, 1.0)
        
        if i < N - 1:
            plt.setp(ax.get_xticklabels(), visible=False)
        else:
            plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)

    out_path = 'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/imgs/'
    os.makedirs(out_path, exist_ok=True)
    fig.savefig(os.path.join(out_path, f'original_{seconds}s_full.png'))

    # plt.show()
    fig.clear()
    plt.close(fig)

## 2D ResNets

In [11]:
plt.style.use('fivethirtyeight') # default, ggplot, fivethirtyeight, classic

for seconds in [60, 30, 10]:
    debug_table = {}
    
    for network in ['ResNet18', 'ResNet50']:
        file_path = f'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/2D_{seconds}s_{network}.txt'
        debug_table[f'2D {network} ({seconds}s)'] = calculate_error_dict(file_path)
    
    fig = plt.figure(num=1, clear=True, figsize=(40.0, 7.5), constrained_layout=True)

    N = len(debug_table)
    for i, (title, error_dict) in enumerate(debug_table.items()):
        ax = fig.add_subplot(N, 1, i + 1)

        for k, v in sorted(error_dict.items()):
            if k != 'accuracy':
                prob = v['errors'] / (v['counts'] * 30)
                rect = ax.bar(k, prob, color=v['color'])
                # ax.bar_label(rect, padding=3)    

        ax.set_title(f'{title} {error_dict["accuracy"]:.2f}%', fontsize=30)
        ax.set_ylim(0.0, 1.)
        
        if i < N - 1:
            plt.setp(ax.get_xticklabels(), visible=False)
        else:
            plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)

    out_path = 'C:/Users/IPIS-Minjae/Desktop/EEG_Project/test_debug/imgs/'
    os.makedirs(out_path, exist_ok=True)
    fig.savefig(os.path.join(out_path, f'2D_{seconds}s.png'))

    # plt.show()
    fig.clear()
    plt.close(fig)